In [7]:
# ---------------------------------------
# IMPORTS
# ---------------------------------------
import time
import csv
from typing import Dict, Any, List
import requests

In [8]:
# ---------------------------------------
# CONFIGURACIÓN LAST.FM
# ---------------------------------------
BASE_URL = "https://ws.audioscrobbler.com/2.0/"
API_KEY = "3bd1dc669ba0fd4ee507954de7dd51a0"

In [9]:
# ---------------------------------------
# FUNCIONES AUXILIARES
# ---------------------------------------
def normalizar_nombre_artista(nombre: str) -> str:
    """
    Normaliza el nombre del artista para comparar con Spotify.
    Pediste: strip + lower + casefold
    """
    if not nombre:
        return ""
    return nombre.strip().lower().casefold()


def lastfm_request(method: str, params: Dict[str, Any], api_key: str = API_KEY, timeout: int = 30) -> Dict[str, Any]:
    """
    Función genérica para llamar a la API de Last.fm y devolver el JSON como diccionario.

    - method: nombre del método/endpoint de last.fm (ej: 'artist.getInfo')
    - params: parámetros específicos del método (ej: {'artist': 'Coldplay'})
    - api_key: la clave de la API
    - timeout: segundos máximos que esperamos antes de cancelar la petición
    """

    # Construimos todos los parámetros que necesita last.fm en una sola variable
    full_params = {
        "method": method,     # Qué método vamos a usar (ej: artist.getInfo)
        "api_key": api_key,   # Nuestra API Key
        "format": "json",     # Queremos la respuesta en formato JSON
        **params              # Aquí metemos el resto de parámetros específicos (artist, autocorrect, etc.)
    }

    # Hacemos la petición GET a la URL base, pasando los parámetros
    r = requests.get(BASE_URL, params=full_params, timeout=timeout)

    # Si el servidor responde con error HTTP (404, 401, 500...), aquí salta una excepción
    r.raise_for_status()

    # Convertimos la respuesta a JSON (en Python: dict/list)
    data = r.json()

    # A veces last.fm devuelve error "dentro" del JSON aunque el HTTP sea 200 (OK)
    if isinstance(data, dict) and "error" in data:
        raise ValueError(f"Last.fm API error {data.get('error')}: {data.get('message')}")

    # Si todo va bien, devolvemos el JSON
    return data


def leer_artistas_csv(ruta_csv: str) -> List[str]:
    """
    Lee un CSV de UNA sola columna (sin cabecera) y devuelve la lista de artistas.

    Importante:
    - El CSV ya viene normalizado, pero por seguridad volvemos a normalizar.
    - Eliminamos duplicados manteniendo el orden (muy importante para no repetir peticiones).
    """

    artistas = []

    # Abrimos el fichero
    with open(ruta_csv, "r", encoding="utf-8") as f:
        lector = csv.reader(f)

        # Cada fila es una lista, pero como solo hay 1 columna, nos interesa row[0]
        for fila in lector:
            if not fila:
                continue  # por si hay líneas vacías

            nombre = normalizar_nombre_artista(fila[0])
            if nombre:
                artistas.append(nombre)

    # Eliminamos duplicados manteniendo el orden
    # Truco simple: dict.fromkeys mantiene el orden en Python 3.7+
    artistas_sin_duplicados = list(dict.fromkeys(artistas))

    return artistas_sin_duplicados


In [10]:
# ---------------------------------------
# FUNCIÓN PRINCIPAL PARA UN ARTISTA
# ---------------------------------------
def get_artist_info(nombre_artista: str, autocorrect: int = 1, timeout: int = 30) -> Dict[str, Any]:
    """
    Pide a la API de Last.fm información de un artista y devuelve un diccionario
    SIEMPRE con las mismas claves.

    - nombre_artista: nombre del artista (normalizado o no)
    - autocorrect: si vale 1, Last.fm intenta corregir nombres mal escritos
    """

    # 1) Llamamos al endpoint/método "artist.getInfo" de Last.fm
    data = lastfm_request(
        "artist.getInfo",
        {"artist": nombre_artista, "autocorrect": autocorrect},
        timeout=timeout
    )

    # 2) Dentro del JSON, la info principal viene en la clave "artist"
    artist = data.get("artist", {}) or {}

    # 3) Dentro de artist, las estadísticas están dentro de la clave "stats"
    stats = artist.get("stats", {}) or {}

    # 4) Sacamos el nombre “raw” (bonito) y el normalizado para cruzar con Spotify
    nombre_raw = artist.get("name", "") or ""
    nombre_norm = normalizar_nombre_artista(nombre_raw) if nombre_raw else normalizar_nombre_artista(nombre_artista)

    # 5) Ojo: listeners y playcount vienen como texto -> lo pasamos a int
    listeners = int(stats.get("listeners", 0) or 0)
    playcount = int(stats.get("playcount", 0) or 0)

    # 6) Devolvemos un diccionario con lo que os interesa para la tabla de artistas
    return {
        "artist_name_raw": nombre_raw,
        "artist_name_norm": nombre_norm,
        "listeners": listeners,
        "playcount": playcount
    }


In [11]:
# ---------------------------------------
# BUCLE DE PETICIONES (MOVIDO A FUNCIÓN)
# ---------------------------------------
def obtener_info_artistas(
    lista_artistas: List[str],
    pausa_segundos: float = 0.25,
    autocorrect: int = 1,
    timeout: int = 30,
    imprimir_cada: int = 25
) -> List[Dict[str, Any]]:
    """
    Hace el bucle de peticiones a Last.fm para una lista de artistas.

    - lista_artistas: lista ya normalizada y sin duplicados
    - pausa_segundos: pausa entre peticiones para evitar rate limit
    - imprimir_cada: cada cuántos artistas imprimimos progreso (para no saturar la salida)
    """

    lista_resultados = []
    inicio = time.time()

    for idx, artista in enumerate(lista_artistas, start=1):
        try:
            info = get_artist_info(artista, autocorrect=autocorrect, timeout=timeout)
            lista_resultados.append(info)

        except Exception as e:
            # Si falla un artista, lo mostramos y seguimos con el siguiente
            print(f"❌ Error con '{artista}': {e}")
            continue

        # Pausa para no saturar la API
        time.sleep(pausa_segundos)

        # Progreso cada X artistas
        if imprimir_cada and idx % imprimir_cada == 0:
            transcurrido = time.time() - inicio
            print(f"✅ {idx}/{len(lista_artistas)} completados | {transcurrido:.0f}s")

    return lista_resultados


def guardar_resultados_csv(resultados: List[Dict[str, Any]], ruta_salida: str) -> None:
    """
    Guarda la lista de diccionarios en CSV.
    Columnas: artist_name_raw, artist_name_norm, listeners, playcount
    """

    cabeceras = ["artist_name_raw", "artist_name_norm", "listeners", "playcount"]

    with open(ruta_salida, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=cabeceras)
        writer.writeheader()
        writer.writerows(resultados)


In [12]:
# ---------------------------------------
# EJEMPLO DE USO (1 GÉNERO)
# ---------------------------------------
# 1) Leemos el CSV de Spotify (una columna, sin cabecera)
#    -> devuelve lista normalizada y sin duplicados
ruta_csv_pop = "Spotify_artistas_pop.csv"
lista_pop = leer_artistas_csv(ruta_csv_pop)

print("Artistas POP (sin duplicados):", len(lista_pop))

# 2) Pedimos a Last.fm listeners y playcount para cada artista
resultados_pop = obtener_info_artistas(
    lista_pop,
    pausa_segundos=0.25,
    autocorrect=1,
    timeout=30,
    imprimir_cada=25
)

# 3) Guardamos el resultado en un CSV final (para no repetir llamadas)
guardar_resultados_csv(resultados_pop, "lastfm_info_artistas_pop.csv")

# 4) Ver los 5 primeros
resultados_pop[:5]

Artistas POP (sin duplicados): 351
✅ 25/351 completados | 22s
✅ 50/351 completados | 43s
✅ 75/351 completados | 64s
✅ 100/351 completados | 85s
✅ 125/351 completados | 112s
✅ 150/351 completados | 143s
✅ 175/351 completados | 163s
✅ 200/351 completados | 184s
✅ 225/351 completados | 207s
✅ 250/351 completados | 227s
✅ 275/351 completados | 249s
✅ 300/351 completados | 269s
✅ 325/351 completados | 290s
✅ 350/351 completados | 312s


[{'artist_name_raw': 'ATEEZ',
  'artist_name_norm': 'ateez',
  'listeners': 768213,
  'playcount': 201000241},
 {'artist_name_raw': 'Aurora',
  'artist_name_norm': 'aurora',
  'listeners': 1754730,
  'playcount': 85891566},
 {'artist_name_raw': 'Abraham Mateo',
  'artist_name_norm': 'abraham mateo',
  'listeners': 162823,
  'playcount': 2578714},
 {'artist_name_raw': 'Adele',
  'artist_name_norm': 'adele',
  'listeners': 5508740,
  'playcount': 311158014},
 {'artist_name_raw': 'Ado',
  'artist_name_norm': 'ado',
  'listeners': 646112,
  'playcount': 71495130}]